In [4]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from datetime import datetime

# Create database engine 
engine = create_engine('sqlite:///example.db', echo=True)

# Create base class for models 
Base = declarative_base()

# Create session factory 
SessionLocal = sessionmaker(bind=engine)

## 2. Creating Models (Database Tables as Python Classes)

In [12]:
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    username = Column(String(50), unique=True, nullable=False)
    email = Column(String(100), unique=True, nullable=False)
    created_at = Column(DateTime, default=datetime.utcnow)

class Post(Base):
    __tablename__ = 'posts'
    
    id = Column(Integer, primary_key=True)
    title = Column(String(200), nullable=False)
    content = Column(String(1000))
    user_id = Column(Integer, ForeignKey('users.id'))
    created_at = Column(DateTime, default=datetime.utcnow)

# Add relationships AFTER both classes are defined
User.posts = relationship("Post", back_populates="author")
Post.author = relationship("User", back_populates="posts")

# Create all tables
Base.metadata.create_all(engine)

/var/folders/wg/8rqwg0nd7995t2tzgq40dsph0000gn/T/ipykernel_5102/3652125552.py:1: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.User, and will be replaced in the string-lookup table.
  class User(Base):


InvalidRequestError: Table 'users' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

## Session Management (Database Transactions)

In [ ]:
# Create a session 
session = SessionLocal()

# Always use Try/finally or context manager 
def database_operation(): 
    session = SessionLocal()
    try: 
       session.commit() 
    except Exception as e: 
        session.rollback()
        raise e
    finally: 
        session.close()

## CRUD Operations 
#### CREATE - Adding Records

In [13]:
def create_user(username, email): 
    with SessionLocal() as session:
        new_user = User(username=username, email=email)
        session.add(new_user)
        session.commit()
        return new_user
    
def create_post(title, content, user_id): 
    with SessionLocal() as session: 
        new_post = Post(title=title, content=content, user_id=user_id)
        session.add(new_post)
        session.commit()
        return new_post
    
    

#### READ - Querying Data

In [14]:
def get_all_users(): 
    with SessionLocal as session: 
        return session.query(User).all()
        
def get_user_by_id(user_id): 
    with SessionLocal as session:
        return session.query(User).filter(User.id == user_id).first()
        
def get_user_by_username(username): 
    with SessionLocal as session:
        return session.query(User).filter(User.username == username).first()
         
def get_posts_by_user(user_id): 
    with SessionLocal as session:
        return session.query(Post).filter(Post.user_id == user_id).all()
        


#### UPDATE 

In [15]:
def update_user_email(user_id, new_email): 
    with SessionLocal as session: 
        user = session.query(User).filter(User.id == user_id).first()
        if user: 
            user.email = new_email
            session.commit()
        return user
    
def update_post_title(post_id, new_title): 
    with SessionLocal() as session: 
        post = session.query(post).filter(post_id==post_id).first()
        if post: 
            post.title = new_title
        return post
         

#### DELETE 

In [16]:
def delete_user(user_id): 
    with SessionLocal() as session: 
        user = session.query(User).filter(User.id == user_id).first()
        if user: 
            session.delete(user)
            session.commit()
            return True
        return False

In [17]:
# Filter with conditions
users_with_gmail = session.query(User).filter(User.email.like('%@gmail.com')).all()

# Multiple conditions
recent_posts = session.query(Post).filter(
    Post.created_at > datetime(2024, 1, 1),
    Post.title.like('%Python%')
).all()

# Ordering
users_by_name = session.query(User).order_by(User.username).all()

# Limiting results
first_10_users = session.query(User).limit(10).all()

# Count records
user_count = session.query(User).count()

InvalidRequestError: One or more mappers failed to initialize - can't proceed with initialization of other mappers. Triggering mapper: 'Mapper[User(users)]'. Original exception was: When initializing mapper Mapper[User(users)], expression 'post' failed to locate a name ('post'). If this is a class name, consider adding this relationship() to the <class '__main__.User'> class after both dependent classes have been defined.

In [18]:
from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from datetime import datetime

# Create database engine
engine = create_engine('sqlite:///example.db', echo=True)

# Create base class
Base = declarative_base()

# Create session factory
SessionLocal = sessionmaker(bind=engine)

# Define User model
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    username = Column(String(50), unique=True, nullable=False)
    email = Column(String(100), unique=True, nullable=False)
    created_at = Column(DateTime, default=datetime.utcnow)

# Define Post model
class Post(Base):
    __tablename__ = 'posts'
    
    id = Column(Integer, primary_key=True)
    title = Column(String(200), nullable=False)
    content = Column(String(1000))
    user_id = Column(Integer, ForeignKey('users.id'))
    created_at = Column(DateTime, default=datetime.utcnow)

# Add relationships AFTER both classes are defined
User.posts = relationship("Post", back_populates="author")
Post.author = relationship("User", back_populates="posts")

# Create all tables
Base.metadata.create_all(engine)

# Test the setup
if __name__ == "__main__":
    # Create a session
    session = SessionLocal()
    
    try:
        # Create a test user
        user = User(username="testuser", email="test@example.com")
        session.add(user)
        session.commit()
        
        # Create a test post
        post = Post(title="My First Post", content="Hello World!", user_id=user.id)
        session.add(post)
        session.commit()
        
        # Query examples
        print("All users:")
        users = session.query(User).all()
        for u in users:
            print(f"  {u.username} - {u.email}")
        
        print("\nAll posts:")
        posts = session.query(Post).all()
        for p in posts:
            print(f"  {p.title} by user {p.user_id}")
        
        print("\nUser with posts:")
        user_with_posts = session.query(User).filter(User.username == "testuser").first()
        if user_with_posts:
            print(f"User: {user_with_posts.username}")
            for post in user_with_posts.posts:
                print(f"  Post: {post.title}")
    
    except Exception as e:
        print(f"Error: {e}")
        session.rollback()
    
    finally:
        session.close()

2025-06-27 16:02:13,147 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-27 16:02:13,148 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2025-06-27 16:02:13,148 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-27 16:02:13,150 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("posts")
2025-06-27 16:02:13,150 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-06-27 16:02:13,151 INFO sqlalchemy.engine.Engine COMMIT
2025-06-27 16:02:13,155 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-27 16:02:13,156 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, created_at) VALUES (?, ?, ?)
2025-06-27 16:02:13,157 INFO sqlalchemy.engine.Engine [generated in 0.00031s] ('testuser', 'test@example.com', '2025-06-27 10:32:13.156805')
2025-06-27 16:02:13,157 INFO sqlalchemy.engine.Engine ROLLBACK
Error: (sqlite3.OperationalError) table users has no column named username
[SQL: INSERT INTO users (username, email, created_at) VALUES (?, ?, ?)]
[parameters: ('testus

/var/folders/wg/8rqwg0nd7995t2tzgq40dsph0000gn/T/ipykernel_5102/3994768051.py:10: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


#### Working with Relationships 

In [19]:
# Get user with thier posts
def get_user_with_posts(user_id): 
    with SessionLocal() as session: 
        user = session.query(User).filter(User.id == user_id).first()
        # Access posts through relationship 
        posts = user.posts
        return user, posts 
 
# Join queries   
def get_posts_with_authors(): 
    with SessionLocal() as session: 
        return session.query(Post).join(User).all()
    

# Filter by related table 
def get_posts_by_username(username): 
    with SessionLocal() as session: 
        return session.query(Post).join(User).filter(User.username == username).all()